# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chimbote,-9.0853,-78.5783,291.41,81,73,2.28,PE,1718023615
1,1,hermanus,-34.4187,19.2345,296.93,47,1,2.45,ZA,1718023615
2,2,kalifornsky,60.4183,-151.2900,284.34,78,100,3.60,US,1718023615
3,3,blackmans bay,-43.0167,147.3167,279.22,86,100,4.43,AU,1718023615
4,4,edinburgh of the seven seas,-37.0676,-12.3116,286.00,64,98,9.80,SH,1718023615


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot_4 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
)
# Display the map
map_plot_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chimbote,-9.0853,-78.5783,291.41,81,73,2.28,PE,1718023615
1,1,hermanus,-34.4187,19.2345,296.93,47,1,2.45,ZA,1718023615
2,2,kalifornsky,60.4183,-151.2900,284.34,78,100,3.60,US,1718023615
3,3,blackmans bay,-43.0167,147.3167,279.22,86,100,4.43,AU,1718023615
4,4,edinburgh of the seven seas,-37.0676,-12.3116,286.00,64,98,9.80,SH,1718023615


In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_n = city_data_df[city_data_df['Cloudiness'] == 0]
city_data_df_na = city_data_df_n[city_data_df_n['Humidity'] <90]
city_data_df_narrow = city_data_df_na[city_data_df_na['Wind Speed'] <4.5]

# Drop any rows with null values
city_data_df_narrow.dropna()

# Display sample data
city_data_df_narrow.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,east london,-33.0153,27.9116,296.67,49,0,2.06,ZA,1718023623
45,45,coquimbo,-29.9533,-71.3436,286.00,87,0,2.06,CL,1718023576
58,58,galliano,29.4422,-90.2992,300.37,83,0,1.54,US,1718023626
75,75,queenstown,-31.8976,26.8753,290.28,27,0,3.22,ZA,1718023629
79,79,arraial do cabo,-22.9661,-42.0278,298.07,73,0,2.57,BR,1718023469


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_narrow.copy()
hotel_df = hotel_df.drop(['City_ID', 'Max Temp', 'Cloudiness', 'Wind Speed', 'Date'], axis=1)
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
43,east london,-33.0153,27.9116,49,ZA,None
45,coquimbo,-29.9533,-71.3436,87,CL,None
58,galliano,29.4422,-90.2992,83,US,None
75,queenstown,-31.8976,26.8753,27,ZA,None
79,arraial do cabo,-22.9661,-42.0278,73,BR,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
east london - nearest hotel: No hotel found
coquimbo - nearest hotel: Hotel Iberia
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
queenstown - nearest hotel: Queens Hotel
arraial do cabo - nearest hotel: No hotel found
longyearbyen - nearest hotel: The Vault
bredasdorp - nearest hotel: Victoria Hotel
alice springs - nearest hotel: Aurora Alice Springs
port elizabeth - nearest hotel: Waterford Hotel
kruisfontein - nearest hotel: Savoy
vila velha - nearest hotel: Hotel Prainha
archangelos - nearest hotel: Semina
rio branco - nearest hotel: No hotel found
superior - nearest hotel: Androy Hotel
lady frere - nearest hotel: Queens Hotel
selinsgrove - nearest hotel: Holiday Inn
lincoln - nearest hotel: New Victorian Suites Lincoln
medina - nearest hotel: The Oberoi Madina
sao vicente - nearest hotel: Pousada Vitória
port alfred - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
amasra - nearest hotel: Liman Hotel
lexington

,City,Lat,Lng,Humidity,Country,Hotel Name
43,east london,-33.0153,27.9116,49,ZA,No hotel found
45,coquimbo,-29.9533,-71.3436,87,CL,Hotel Iberia
58,galliano,29.4422,-90.2992,83,US,Fairfield Inn & Suites by Marriott Cut Off-Gal...
75,queenstown,-31.8976,26.8753,27,ZA,Queens Hotel
79,arraial do cabo,-22.9661,-42.0278,73,BR,No hotel found
106,longyearbyen,78.2186,15.6401,61,SJ,The Vault
127,bredasdorp,-34.5322,20.0403,47,ZA,Victoria Hotel
129,alice springs,-23.7000,133.8833,81,AU,Aurora Alice Springs
135,port elizabeth,-33.9180,25.5701,41,ZA,Waterford Hotel
152,kruisfontein,-34.0033,24.7314,40,ZA,Savoy


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display
map_plot_4 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    hover_cols=['Hotel Name', 'Country']
    )


# Display the map
map_plot_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)